In [2]:
print("Notebook is working")

Notebook is working


## Imports

In [8]:
import requests
from datetime import datetime, time, timezone, date as dt_date

## Stadium and game information

In [4]:
# Braves (Truist Park)
stadium_name = "Truist Park"
latitude = 33.8908
longitude = -84.4678
team_id = 144
date = dt_date.today().isoformat()

In [33]:
# Get the weather data for today's date and Citizens Bank Park
weather_data = get_weather_data(latitude, longitude, date)

# Show the top-level keys of the response
print(weather_data.keys())


dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly'])


In [35]:
game_time_data = filter_for_game_time(weather_data)

for entry in game_time_data:
    print(f"Time: {entry['time']}, Direction: {entry['direction_deg']}°, "
          f"Speed: {entry['speed_kmh']} km/h ({entry['speed_mph']} mph)")


Time: 2025-07-03T19:00, Direction: 257°, Speed: 19.2 km/h (11.93 mph)
Time: 2025-07-03T20:00, Direction: 218°, Speed: 15.1 km/h (9.38 mph)
Time: 2025-07-03T21:00, Direction: 349°, Speed: 19.4 km/h (12.05 mph)
Time: 2025-07-03T22:00, Direction: 11°, Speed: 7.7 km/h (4.78 mph)


In [7]:
# 1. Call MLB schedule API for the Braves (team ID 144) for today's date
mlb_url = "https://statsapi.mlb.com/api/v1/schedule"
mlb_params = {
    "sportId": 1,
    "date": date,     # today's e variable you've set
    "teamId": 144     # Braves team ID
}
mlb_resp = requests.get(mlb_url, params=mlb_params)
mlb_data = mlb_resp.json()

games = mlb_data.get("dates", [])


In [6]:
if games:
    game = games[0]["games"][0]  # First Braves game today
    utc_start = game["gameDate"]  # e.g. "2025-07-01T23:20:00Z"
    print("Raw UTC start time:", utc_start)
else:
    print("No Braves game scheduled today.")


Raw UTC start time: 2025-07-04T23:15:00Z


In [9]:
# Convert UTC time to your local time zone
dt_utc = datetime.fromisoformat(utc_start.replace("Z", "+00:00"))
dt_local = dt_utc.astimezone(tz=None)  # Auto-detects your system's local time
game_hour = dt_local.hour

print("Local start time:", dt_local)
print("Start hour only (for filtering):", game_hour)


Local start time: 2025-07-04 19:15:00-04:00
Start hour only (for filtering): 19


In [10]:
# Open-Meteo API
weather_url = "https://api.open-meteo.com/v1/forecast"
weather_params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": date,
    "end_date": date,
    "hourly": "winddirection_10m,windspeed_10m",
    "timezone": "America/New_York"
}

weather_resp = requests.get(weather_url, params=weather_params)
weather_data = weather_resp.json()


In [15]:
def filter_for_game_time(data, start_hour, end_hour):
    hourly = data["hourly"]
    times = hourly["time"]
    directions = hourly["winddirection_10m"]
    speeds_kmh = hourly["windspeed_10m"]

    filtered = []

    for i, t_str in enumerate(times):
        dt = datetime.strptime(t_str, "%Y-%m-%dT%H:%M")
        if start_hour <= dt.hour <= end_hour:
            filtered.append({
                "time": t_str,
                "direction_deg": directions[i],
                "speed_kmh": speeds_kmh[i],
                "speed_mph": round(speeds_kmh[i] * 0.621371, 2)
            })

    return filtered


In [17]:
if game_hour is not None:
    filtered_wind = filter_for_game_time(weather_data, start_hour=game_hour, end_hour=game_hour+3)

    for entry in filtered_wind:
        print(f"Time: {entry['time']}, Direction: {entry['direction_deg']}°, "
              f"Speed: {entry['speed_kmh']} km/h ({entry['speed_mph']} mph)")
else:
    print("No game today — no weather filtering.")


Time: 2025-07-04T19:00, Direction: 48°, Speed: 9.7 km/h (6.03 mph)
Time: 2025-07-04T20:00, Direction: 45°, Speed: 9.2 km/h (5.72 mph)
Time: 2025-07-04T21:00, Direction: 113°, Speed: 12.1 km/h (7.52 mph)
Time: 2025-07-04T22:00, Direction: 49°, Speed: 3.3 km/h (2.05 mph)
